In [1]:
# import PIL
# # from keras import models
# from keras import layers
# from tensorflow.keras import optimizers
# import os
# import glob
# import shutil
# import sys
# import numpy as np
# from skimage.io import imread
# import matplotlib.pyplot as plt
# import os
# from tensorflow.keras import callbacks
# import pandas as pd
# from efficientnet.keras import EfficientNetB5 as Net
# from efficientnet.keras import center_crop_and_resize, preprocess_input
# from tensorflow.keras.models import load_model
# import tensorflow.keras as keras
# from keras import models
# from tensorflow.keras.models import Model, model_from_json
# import tensorflow as tf
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense#, Conv2D, BatchNormalization
# from tensorflow.keras.applications import EfficientNetB5

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# from PIL import Image, ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True


In [1]:
# ===============================
# Core
# ===============================
import os
import sys
import glob
import shutil
import numpy as np
import pandas as pd

# ===============================
# Image
# ===============================
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from skimage.io import imread
import matplotlib.pyplot as plt

# ===============================
# TensorFlow / Keras (USE THIS ONLY)
# ===============================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.models import Model, load_model, model_from_json
from tensorflow.keras.layers import (
    Input,
    GlobalAveragePooling2D,
    Dropout,
    Dense
)

# ===============================
# EfficientNet (USE TF VERSION)
# ===============================
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.applications.efficientnet import preprocess_input

# ===============================
# GPU
# ===============================
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
print(tf.__version__)      # 2.6.0rc0
print(keras.__version__)   # 2.6.0rc0
print(tf.config.list_physical_devices('GPU'))


2.6.0-rc0
2.6.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout

class FixedDropout(Dropout):
    def __init__(self, rate, **kwargs):
        super(FixedDropout, self).__init__(rate, **kwargs)

    def call(self, inputs, training=None):
        if training is None:
            training = False
        return super().call(inputs, training=training)
    
from tensorflow.keras.models import model_from_json

with open('/media/tohn/HDD2/Model_unlearn/Models_USAI/ML-unlearn/R2/unfreezeB4-B7/models/modelEffNetB5_ML-unlearn_unfreezeB4-B7-R2.json', 'r') as f:
    loaded_model_json = f.read()

model = model_from_json(
    loaded_model_json,
    custom_objects={'FixedDropout': FixedDropout}
)

model.load_weights(
    "/media/tohn/HDD2/Model_unlearn/Models_USAI/ML-unlearn/R2/unfreezeB4-B7/models/modelEffNetB5_ML-unlearn_unfreezeB4-B7-R2.weights.h5"
)

print("Loaded model successfully ✅")



Loaded model successfully ✅


In [4]:
height = width = model.input_shape[1]
x = model.get_layer('global_average_pooling2d').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
new_input (InputLayer)       [(None, 456, 456, 3)]     0         
_________________________________________________________________
efficientnet-b5 (Functional) (None, None, None, 2048)  28513520  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
prediction_layer (Dense)     (None, 15)                30735     
Total params: 28,544,255
Trainable params: 26,859,743
Non-trainable params: 1,684,512
_________________________________________________________________


## 🌷 Models loading 🌷 


In [10]:
# # # 🍉 load json and create model --------------------

# json_file = open('/media/tohn/HDD2/Model_unlearn/Models_USAI/ML-unlearn/R2/unfreezeB4-B7/models/modelEffNetB5_ML-unlearn_unfreezeB4-B7-R2.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# model = model_from_json(loaded_model_json)

# # # 🍉 load weights into new model  -------------------

# model.load_weights("/media/tohn/HDD2/Model_unlearn/Models_USAI/ML-unlearn/R2/unfreezeB4-B7/models/modelEffNetB5_ML-unlearn_unfreezeB4-B7-R2.weights.h5")
# print("Loaded model from disk")

# height = width = model.input_shape[1]

# x = model.get_layer('global_average_pooling2d').output
# prediction_layer = model.output
# model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

# model.summary()

## Train set ➖🎄

In [11]:
import pandas as pd
train = '/media/tohn/HDD/VISION_dataset/USAI/Data_CCA_6569.csv'
# test = '/home/yupaporn/codes/USAI/Testdf.csv'
df0 = pd.read_csv (train)

print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
dataframe = df0[df0['Spilt'] == 'Train']
dataframe = dataframe.reset_index(drop=True)
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)

(6569, 27)
(5257, 27)
Normal:  (3434, 27)
Abnormal:  (1823, 27)


In [12]:
# dataframe

In [13]:
base_dir = '/media/tohn/SSD/Images/Image1'
os.chdir(base_dir)
train_dir = os.path.join(base_dir, 'train')

validation_dir = os.path.join(base_dir, 'validation')

In [16]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Sub_class_New',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 5257 validated image filenames belonging to 15 classes.
{0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}


In [7]:
# dataframe

In [8]:
dataframe.columns

Index(['Unnamed: 0', 'Case', 'Abs Position', 'Sub Position', 'Class',
       'Path Full', 'Path Crop', 'Views', 'fold', '15AB', 'originalImage',
       'left', 'top', 'width', 'height', 'Rleft', 'Rtop', 'Rwidth', 'Rheight',
       'Sub_class_New', 'tag_AjNit', 'tag_AjWan', 'Test150', 'Spilt',
       'Sub_Position_New', 'filename', 'Sub_Position_Label'],
      dtype='object')

In [9]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

labels =  {0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}

pred_list,  prob_list, test_list, vectorlist = [], [], [], []
for img in range(len(dataframe)):
    print(f'on process----{1+img}/{len(dataframe)-(1+img)}')
    result1, result2 = predict_image(dataframe['Path Crop'][img], model2)
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
# keep information
vector_array = np.array(vectorlist)
dataset = pd.DataFrame(vector_array[:, :])

# dataset['effnet_Predict'] = pred_list
# dataset['effnet_Prob'] = prob_list
# dataset['Case'] = dataframe['Case'] 
# dataset['Sub_class'] = dataframe['Sub_class'] 
# dataset['Path Full'] = dataframe['Path Full']
# dataset['Class'] = dataframe['Class'] 
# dataset['fold'] = dataframe['fold'] 
# dataset['Path Crop'] = dataframe['Path Crop']
# dataset['Views'] = dataframe['Views']
# dataset['tagName'] = dataframe['tagName']
# dataset['filename'] = dataframe['filename']
# dataset['Sub_Position_New'] = dataframe['Sub_Position_New']
# dataset['Sub_Position_Label'] = Sub_Position_Label['tagName']
# print(dataset.shape)
# dataset.head()

on process----1/5256
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
on process----2/5255
on process----3/5254
on process----4/5253
on process----5/5252
on process----6/5251
on process----7/5250
on process----8/5249
on process----9/5248
on process----10/5247
on process----11/5246
on process----12/5245
on process----13/5244
on process----14/5243
on process----15/5242
on process----16/5241
on process----17/5240
on process----18/5239
on process----19/5238
on process----20/5237
on process----21/5236
on process----22/5235
on process----23/5234
on process----24/5233
on process----25/5232
on process----26/5231
on process----27/5230
on process----28/5229
on process----29/5228
on process----30/5227
on process----31/5226
on process----32/5225
on process----33/5224
on process----34/5223
on process----35/5222
on process----36/5221
on process----37/5220
on process----38/5219
on process----39/5218
on process---

In [10]:
dataframe.columns

Index(['Unnamed: 0', 'Case', 'Abs Position', 'Sub Position', 'Class',
       'Path Full', 'Path Crop', 'Views', 'fold', '15AB', 'originalImage',
       'left', 'top', 'width', 'height', 'Rleft', 'Rtop', 'Rwidth', 'Rheight',
       'Sub_class_New', 'tag_AjNit', 'tag_AjWan', 'Test150', 'Spilt',
       'Sub_Position_New', 'filename', 'Sub_Position_Label'],
      dtype='object')

In [11]:
dataset['effnet_Predict'] = pred_list
dataset['effnet_Prob'] = prob_list
dataset['Case'] = dataframe['Case'] 
dataset['Sub_class_New'] = dataframe['Sub_class_New'] 
dataset['Path Full'] = dataframe['Path Full']
dataset['Class'] = dataframe['Class'] 
dataset['fold'] = dataframe['fold'] 
dataset['Path Crop'] = dataframe['Path Crop']
dataset['Views'] = dataframe['Views']
dataset['15AB'] = dataframe['15AB']
dataset['filename'] = dataframe['filename']
dataset['Sub_Position_New'] = dataframe['Sub_Position_New']
dataset['Sub_Position_Label'] = dataframe['Sub_Position_Label']
print(dataset.shape)
dataset.head()

(5257, 2061)


,0,1,2,3,4,5,6,7,8,9,...,Sub_class_New,Path Full,Class,fold,Path Crop,Views,15AB,filename,Sub_Position_New,Sub_Position_Label
0,0.030133,0.000557,-0.029769,0.347194,-0.139205,0.166534,0.907789,0.170190,0.407148,-0.060172,...,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,Abnormal,3,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,MildFattyLiver,AB01 P3-1 C015.JPG,P3,P3
1,-0.092604,-0.082832,0.102105,0.581072,-0.146944,0.298722,0.953581,0.073126,0.946436,-0.008451,...,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,Abnormal,3,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,MildFattyLiver,AB01 P1 C015.JPG,P1,P1
2,-0.047872,-0.026505,0.500955,0.671585,-0.155412,0.222590,0.756759,0.167506,0.575030,0.191692,...,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,Abnormal,3,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,MildFattyLiver,AB01 P2 C015.JPG,P2,P2
3,-0.008266,0.029715,0.083367,0.655447,-0.156899,0.316843,0.845403,0.210877,0.778528,0.184029,...,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,Abnormal,3,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,MildFattyLiver,AB01 P4-2 C015.JPG,P5,P5
4,0.089700,-0.058428,-0.073501,0.327241,-0.198723,0.498390,0.676407,0.316914,0.228938,0.168227,...,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,Abnormal,3,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,MildFattyLiver,AB01 P4-1 C015.JPG,P4,P4


In [12]:
# save CSV
dataset.to_csv(f'/home/yupaporn/codes/CCA-USAI-2026/FV/FeaturesVector_Traindf_unlearn_block4_block7.csv')
# dataset.to_csv(f'//home/yupaporn/codes/CCA-USAI-2026/FV/FeaturesVector_Testdf_unlearn_block4_block7.csv')

In [13]:
# dataset

## Test set ➖🎄

In [14]:
import pandas as pd
test = '/media/tohn/HDD/VISION_dataset/USAI/Data_CCA_6569.csv'
# test = '/home/yupaporn/codes/USAI/Testdf.csv'
df0 = pd.read_csv (test)


print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
dataframe = df0[df0['Spilt'] != 'Train']
dataframe = dataframe.reset_index(drop=True)
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)

(6569, 27)
(1312, 27)
Normal:  (857, 27)
Abnormal:  (455, 27)


In [15]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

labels =  {0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}

pred_list,  prob_list, test_list, vectorlist = [], [], [], []
for img in range(len(dataframe)):
    print(f'on process----{1+img}/{len(dataframe)-(1+img)}')
    result1, result2 = predict_image(dataframe['Path Crop'][img], model2)
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
# keep information
vector_array = np.array(vectorlist)
dataset = pd.DataFrame(vector_array[:, :])

dataset['effnet_Predict'] = pred_list
dataset['effnet_Prob'] = prob_list
dataset['Case'] = dataframe['Case'] 
dataset['Sub_class_New'] = dataframe['Sub_class_New'] 
dataset['Path Full'] = dataframe['Path Full']
dataset['Class'] = dataframe['Class'] 
dataset['fold'] = dataframe['fold'] 
dataset['Path Crop'] = dataframe['Path Crop']
dataset['Views'] = dataframe['Views']
dataset['15AB'] = dataframe['15AB']
dataset['filename'] = dataframe['filename']
dataset['Sub_Position_New'] = dataframe['Sub_Position_New']
dataset['Sub_Position_Label'] = dataframe['Sub_Position_Label']
print(dataset.shape)
dataset.head()

on process----1/1311
on process----2/1310
on process----3/1309
on process----4/1308
on process----5/1307
on process----6/1306
on process----7/1305
on process----8/1304
on process----9/1303
on process----10/1302
on process----11/1301
on process----12/1300
on process----13/1299
on process----14/1298
on process----15/1297
on process----16/1296
on process----17/1295
on process----18/1294
on process----19/1293
on process----20/1292
on process----21/1291
on process----22/1290
on process----23/1289
on process----24/1288
on process----25/1287
on process----26/1286
on process----27/1285
on process----28/1284
on process----29/1283
on process----30/1282
on process----31/1281
on process----32/1280
on process----33/1279
on process----34/1278
on process----35/1277
on process----36/1276
on process----37/1275
on process----38/1274
on process----39/1273
on process----40/1272
on process----41/1271
on process----42/1270
on process----43/1269
on process----44/1268
on process----45/1267
on process----46/12

,0,1,2,3,4,5,6,7,8,9,...,Sub_class_New,Path Full,Class,fold,Path Crop,Views,15AB,filename,Sub_Position_New,Sub_Position_Label
0,0.075308,0.579924,0.863569,0.392704,0.536715,0.744365,0.339434,0.432817,0.620449,0.210296,...,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,Abnormal,2,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,MildFattyLiver,AB01 P1 C040.JPG,P1,P1
1,0.175207,0.420301,0.157335,0.486010,-0.004742,0.116420,1.101452,0.408028,0.500560,0.145058,...,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,Abnormal,2,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,MildFattyLiver,AB01 P2 C040.JPG,P2,P2
2,0.080880,0.135601,0.380881,0.771338,-0.089207,0.467602,1.119100,-0.137553,1.217198,0.378177,...,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,Abnormal,2,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,MildFattyLiver,AB01 P4-1 C040.JPG,P4,P4
3,-0.052900,0.213714,0.071093,0.373811,-0.000477,0.110895,0.619053,-0.036836,0.177048,0.014405,...,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,Abnormal,2,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-C,MildFattyLiver,AB01 P5-1 C040.JPG,P6,P6
4,0.117390,0.656421,0.662734,0.987979,-0.062307,0.104385,0.733278,0.158326,0.445317,0.200059,...,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,Abnormal,2,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,MildFattyLiver,AB01 P3-1 C040.JPG,P3,P3


In [16]:
dataframe.columns

Index(['Unnamed: 0', 'Case', 'Abs Position', 'Sub Position', 'Class',
       'Path Full', 'Path Crop', 'Views', 'fold', '15AB', 'originalImage',
       'left', 'top', 'width', 'height', 'Rleft', 'Rtop', 'Rwidth', 'Rheight',
       'Sub_class_New', 'tag_AjNit', 'tag_AjWan', 'Test150', 'Spilt',
       'Sub_Position_New', 'filename', 'Sub_Position_Label'],
      dtype='object')

In [17]:
# save CSV
# dataset.to_csv(f'/home/yupaporn/codes/CCA-USAI-2026/FV/FeaturesVector_Traindf_unlearn_block4_block7.csv')
dataset.to_csv(f'/home/yupaporn/codes/CCA-USAI-2026/FV/FeaturesVector_Testdf_unlearn_block4_block7.csv')

In [45]:
dataset.columns

Index([                   0,                    1,                    2,
                          3,                    4,                    5,
                          6,                    7,                    8,
                          9,
       ...
              'effnet_Prob',               'Case',          'Path Full',
                    'Class',               'fold',          'Path Crop',
                    'Views',           'filename',   'Sub_Position_New',
       'Sub_Position_Label'],
      dtype='object', length=2059)

In [21]:
set(dataframe['Sub_class_New'])

{'AB01',
 'AB02',
 'AB03',
 'AB04',
 'AB05',
 'AB06',
 'AB07',
 'AB081',
 'AB082',
 'AB083',
 'AB09',
 'AB10',
 'AB11',
 'AB12',
 'Normal'}

In [ ]:
# "Sub_class_New"
# "Sub_Position_Label"
# 'Path Crop'
# 'Unlearn_pred_top1'
# 'Unlearn_pred_top2'
# 'Unlearn_pred_top3'
# 'Unlearn_pred_top4'
# 'Unlearn_pred_top5'
# 'Unlearn_prob_all'